<a href="https://colab.research.google.com/github/tatsuyaokunaga/CNN-scratch/blob/master/visualize_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualize Images

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
%matplotlib inline

In [0]:
import pip
installed_packages=pip.get_installed_distributions()
installed_packages_list = sorted(["%s==%s" % (i.key,i.version) for i in installed_packages])
print(installed_packages_list)

In [0]:

# ローカルファイルを直接アップロードする
from google.colab import files
uploaded = files.upload()


In [0]:
# データの中身を確認
# example = next(tf.python_io.tf_record_iterator(OUTPUT_TFRECORD_NAME))
# tf.train.Example.FromString(example)

In [0]:
import os
path = "./"
file_path = [os.path.join(path, 'training10_{0}/training10_{0}.tfrecords'.format(i)) for i in range(5)]

In [0]:
# データ数カウント
cnts=0
for i, p in enumerate(file_path):
    cnt = len(list(tf.python_io.tf_record_iterator(p)))
    print("NO:{}, データ件数：{}".format(i, cnt))
    cnts+=cnt
print("合計データ件数：{}".format(cnts))

In [0]:
def  convert_tf(record):
        example = tf.train.Example()
        example.ParseFromString(record)  # バイナリデータからの読み込み

        label = example.features.feature["label_normal"].int64_list.value[0]
        image = example.features.feature["image"].bytes_list.value[0]

        image = np.fromstring(image, dtype=np.uint8)
        image = image.reshape([height, width])
        img = Image.fromarray(image, "L")  # グレースケール
        return img, label

In [0]:
# TFRecordファイルを画像に戻す
height, width = (299,299)
label_list=[]
count = 0
for i in range(len(file_path)-1):
    for record  in tf.python_io.tf_record_iterator(file_path[i]):
        # trainデータ
        img,label = convert_tf(record)
        label_list.append(label)
        img.save(os.path.join("images/train", "img{0}-{1}.jpg".format(str(count), label)))
        count += 1
labels=np.array(label_list)
np.save('./images/y_train.npy',labels)

In [0]:
count = 0
label_list=[]
for record  in tf.python_io.tf_record_iterator(file_path[4]):
    # trainデータ
    img,label = convert_tf(record)
    
    label_list.append(label)    
    img.save(os.path.join("images/test", "img{0}-{1}.jpg".format(str(count), label)))
    count += 1
    
labels=np.array(label_list)
np.save('./images/y_test.npy',labels)

In [0]:
del labels


In [0]:
import gc
gc.collect()

0

In [0]:
height, width = (299,299)

image_list=[]

for file in os.listdir('./images/train'):
    if file == '.DS_Store':
        continue
    file_path = './images/train' + '/' + file
    image=Image.open(file_path)
    image = image.resize((height, width))
    image=np.asarray(image).astype('float32')
    image_list.append(image/255.0) # 正規化
    # ndarrayに変換
X_train = np.array(image_list)
np.save('./images/X_train.npy',X_train)
del X_train

In [0]:
image_list=[]
for file in os.listdir('./images/test'):
    if file == '.DS_Store':
        continue
    file_path = './images/test' + '/' + file
    image=Image.open(file_path)
    image = image.resize((height, width))
    
    image=np.asarray(image).astype('float32')
    
    image_list.append(image/255.0) # 正規化
    # ndarrayに変換
X_test  = np.array(image_list)
np.save('./images/X_test.npy',X_test)
del X_test

### データセットのload

In [0]:
X_train=np.load('./images/X_train.npy')
X_test=np.load('./images/X_test.npy')

In [0]:
from keras.utils.np_utils import to_categorical
nb_classes = 2
y_train =to_categorical(np.load('./images/y_train.npy'), nb_classes)
y_test =to_categorical(np.load('./images/y_test.npy'), nb_classes)

In [0]:
from keras import backend as K
from keras.models import Sequential
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.losses import categorical_crossentropy
from keras import Sequential
from keras.layers import Dense, Activation,Dropout,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam , SGD, RMSprop
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

訓練データは、training10_0-3、テストデータはtraining10_4を使用する。(kaggleのkernelより)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25)

In [0]:
# 299 x 299の画像がgrayscaleで1chなので、299, 299, 1にreshapeする
X_train = X_train.reshape(X_train.shape[0], 299, 299, 1)
X_valid = X_valid.reshape(X_valid.shape[0], 299, 299, 1)
X_test = X_test.reshape(X_test.shape[0], 299, 299, 1)

In [0]:
from keras import layers
from keras import models
 # CNNモデルの構築
model = Sequential()

model.add(Conv2D(32,(3,3),padding='same',input_shape=(299,299,1)))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Conv2D(64,3,padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='binary_crossentropy',optimizer='RMSprop',
             metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 299, 299, 32)      320       
_________________________________________________________________
activation_19 (Activation)   (None, 299, 299, 32)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 297, 297, 32)      9248      
_________________________________________________________________
activation_20 (Activation)   (None, 297, 297, 32)      0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 297, 297, 32)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 297, 297, 64)      18496     
_________________________________________________________________
activation_21 (Activation)   (None, 297, 297, 64)      0         
__________

In [0]:
# 学習
history=model.fit(X_train,y_train,batch_size=256,epochs=10,verbose=1,validation_data=(X_valid, y_valid))

Train on 33531 samples, validate on 11177 samples
Epoch 1/10
 4864/33531 [===>..........................] - ETA: 21:10:51 - loss: 1.9909 - acc: 0.8705

KeyboardInterrupt: 

In [0]:
#評価
score =model.evaluate(X_test,y_test,verbose =0)
print('Test loss:',score[0])
print('Test accuracy:',score[1])